In [6]:
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import cv2
import matplotlib.pyplot as plt

def escalar_imagen(image, max_width):
  original_height, original_width = image.shape
  ratio = max_width / original_width
  height = int(original_height * ratio)
  return cv2.resize(image, (max_width, height))

def mostrar_imagen(image, label, max_width):
  imagen_resized = escalar_imagen(image, max_width)
  imagen_tk = ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
  label.config(image=imagen_tk)
  label.image = imagen_tk

def suavizado(n):
    n=int(n)/20
    if opcion_filtro.get() == "filtro_ideal":
        H_butter, espectro, g_real = filtro_ideal(n)
    else:
        H_butter, espectro, g_real = filtro_alto(n)
    mostrar_imagen(H_butter,label_butter,300)
    mostrar_imagen(espectro,label_matriz,300)
    mostrar_imagen(g_real, label_g_real,300)

def filtro_ideal(valor):
    
    fil, col = img.shape

    img_padded = np.pad(img,((0,fil),(0,col)),'constant',constant_values=((0, 0),(0,0)))

    f, c = np.ogrid[0:2*fil,0:2*col]

    n = valor
    D =  np.sqrt( (f-fil)**2 + (c-col)**2 )
    D0 = (0.12*fil)
    H_pb_butter = (1/(1 + (D/D0)**(2*n)))
    H_butter= H_pb_butter*255/np.max(H_pb_butter)

    img_fft = np.fft.fft2(img_padded)
    img_fft_shift = np.fft.fftshift(img_fft)

    G_fft = img_fft_shift * H_pb_butter
    espectro = np.log(1 + np.abs(G_fft))
    espectro =espectro*255/np.max(espectro)
    #plt.imshow(espectro, cmap='gray')
    #print(np.min(espectro))
    #print(np.max(espectro))
    #plt.show()

    G_fft_ishift = np.fft.ifftshift(G_fft)
    g = np.fft.ifft2(G_fft_ishift)
    g_real = np.real(g)[0:fil, 0:col]
    
    return H_butter,espectro, g_real

def filtro_alto(valor):
    fil, col = img.shape

    img_padded = np.pad(img,((0,fil),(0,col)),'constant',constant_values=((0, 0),(0,0)))

    f, c = np.ogrid[0:2*fil,0:2*col]

    n = valor
    D =  np.sqrt( (f-fil)**2 + (c-col)**2 )
    D0 = (0.1*fil)
    H_pa_butter = 1 - 1/(1 + (D/D0)**(2*n))
    H_butter= H_pa_butter*255/np.max(H_pa_butter)

    img_fft = np.fft.fft2(img_padded)
    img_fft_shift = np.fft.fftshift(img_fft)

    G_fft = img_fft_shift * H_pa_butter
    espectro = np.log(1 + np.abs(G_fft))
    espectro =espectro*255/np.max(espectro)

    G_fft_ishift = np.fft.ifftshift(G_fft)
    g = np.fft.ifft2(G_fft_ishift)
    g_real = np.real(g)[0:fil, 0:col]
    
    return H_butter,espectro, g_real
    
image_path = "joker-heath-ledger-1570108200.jpg" # Ruta de la imagen a mostrar
img = cv2.imread(image_path, 0)

ventana = tk.Tk()
ventana.title("Mostrar Imagen y Matriz con Tkinter")

label_imagen = tk.Label(ventana)
mostrar_imagen(img, label_imagen, 300)
label_imagen.grid(row=0, column=0, padx=10, pady=10)

label_butter = tk.Label(ventana)
label_butter.grid(row=0, column=1, padx=10, pady=10)

label_matriz = tk.Label(ventana)
label_matriz.grid(row=0, column=2, padx=10, pady=10)

label_g_real = tk.Label(ventana)
label_g_real.grid(row=0, column=3, padx=10, pady=10)
slider_n = tk.Scale(ventana, from_=1, to=20, orient=tk.HORIZONTAL, label="Valor de n", command=suavizado)
slider_n.set(2)
slider_n.grid(row=1, column=0, columnspan=3, pady=10)

opcion_filtro = tk.StringVar()

rb_filtro_ideal = tk.Radiobutton(ventana, text="Filtro Ideal", variable=opcion_filtro, value="filtro_ideal")
rb_filtro_ideal.grid(row=2, column=0, pady=5)

rb_filtro_alto = tk.Radiobutton(ventana, text="Filtro Alto", variable=opcion_filtro, value="filtro_alto")
rb_filtro_alto.grid(row=2, column=1, pady=5)

opcion_filtro.set("filtro_ideal")

ventana.mainloop()